In [11]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [12]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

In [13]:
load_dotenv()

True

In [14]:
api_key = os.getenv('EPA_API_KEY')

In [15]:
def fetch_aqi_data(year, api_key):
    url = 'https://aqs.epa.gov/data/api/annualData/byCBSA'
    params = {
        'email': 'massmediafan@gmail.com',
        'key': api_key,
        'param': '88101',  # Column for PM2.5
        'bdate': f'{year}0101',
        'edate': f'{year}1231',
        'cbsa': cbsa  # CBSA code for Tulsa, LA and NY define later
    }
    response = requests.get(url, params=params)
    data = response.json()


In [21]:
api_key = os.getenv('EPA_API_KEY')

In [16]:
# Range of years
years = range(2014, 2024)  # Adjust this range as needed

In [22]:
# CBSA codes
cbsa_codes = {
    'Tulsa': '35620', 
    'Los Angeles': '31080',
    'New York': '46140'
}

In [28]:
def fetch_aqi_data(year, api_key, cbsa_code):
    # EmptyDataFrame to avoid NoneType errors
    
    return pd.DataFrame()  # Temporary placeholder

# Concatenate data for the years and CBSA codes
all_data = pd.DataFrame()

for cbsa_name, cbsa_code in cbsa_codes.items():
    for year in years:
        yearly_data = fetch_aqi_data(year, api_key, cbsa_code)
        if not yearly_data.empty:
            yearly_data['CBSA'] = cbsa_name  # Add a column for CBSA name
            all_data = pd.concat([all_data, yearly_data])

In [18]:
# Reset the index
all_data.reset_index(drop=True, inplace=True)

In [19]:
# Convert first max datetime for dates
all_data['first_max_datetime'] = pd.to_datetime(all_data['first_max_datetime'])

In [20]:
# Set date column as the index
all_data.set_index('first_max_datetime', inplace=True)

In [10]:
# View DataFrame
print(all_data.head())
print(all_data.tail())

                   state_code county_code site_number parameter_code  poc  \
first_max_datetime                                                          
2014-02-01                 36         103        0002          88101    1   
2014-02-01                 36         103        0002          88101    1   
2014-02-01                 36         103        0002          88101    1   
2014-02-01                 36         103        0002          88101    1   
2014-02-01                 36         103        0002          88101    1   

                    latitude  longitude  datum                 parameter  \
first_max_datetime                                                         
2014-02-01          40.74529  -73.41919  WGS84  PM2.5 - Local Conditions   
2014-02-01          40.74529  -73.41919  WGS84  PM2.5 - Local Conditions   
2014-02-01          40.74529  -73.41919  WGS84  PM2.5 - Local Conditions   
2014-02-01          40.74529  -73.41919  WGS84  PM2.5 - Local Conditions   
2014